In [15]:
!pip install pygame

In [16]:
import pygame
import math
from queue import PriorityQueue

In [17]:
WIDTH = 750
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("A* Path Finding Algorithm")
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 255, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165, 0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)

In [18]:
all_cells = {}
neighbors = {}
final_path =[]

In [19]:
def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width))

In [20]:
def draw(win, all_cells, rows, width):
    gap = width //rows
    win.fill(WHITE)

    for item in all_cells:
        pygame.draw.rect(win, all_cells[item], (item[0] * gap, item[1] * gap, gap, gap))

    draw_grid(win, rows, width)

    pygame.display.update()

In [21]:
def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos

    row = y // gap
    col = x // gap

    return row, col

In [22]:
def collision_checker(row,col,all_cells):

    if row < 0 or col < 0:
        return True

    if row > 50 - 1 or col > 50 - 1:
        return True

    return all_cells[(row,col)] == BLACK

In [23]:
def update_neighbors(row,col,oc):
    temp_neighbors = []

    if not collision_checker(row + 1, col,oc):  # DOWN
        temp_neighbors.append([row + 1,col])

    if not collision_checker(row - 1, col,oc):  # UP
        temp_neighbors.append( [row - 1,col])

    if not collision_checker(row, col + 1,oc):  # RIGHT
        temp_neighbors.append( [row,col + 1])

    if not collision_checker(row, col - 1,oc):  # LEFT
        temp_neighbors.append( [row,col - 1])

    neighbors[row,col] = temp_neighbors

In [24]:
def heu(x1, x2, y1, y2):
    return abs(x1 - x2) + abs(y1 - y2)

In [25]:
def a_star_algorithm(draw,grid, start, goal, height, width):

    # a priority queue would help us to always pick the smallest a star value from the first element
    all_a_star_score = PriorityQueue()

    # heuristic list to keep track of all grid's updated heuristic
    heuristics = {}

    # list that states all grids' a star score
    a_star_score_list = {}

    # assign heuristics to all grids
    for i in range(width):
        for w in range(height):
            heuristics[(i, w)] = 0

    # set all grids' a star score to infinity
    for i in range(width):
        for w in range(height):
            a_star_score_list[(i, w)] = float("inf")
    # assign start node's a star score as 0
    a_star_score_list[start] = 0

    # initialize the queue
    all_a_star_score.put(0 + heuristics[(start[0], start[1])])
    all_a_star_score_dic = {(start, start): 0 + heuristics[(start[0], start[1])]}
    loop = True

    while all_a_star_score.not_empty and loop:

        # find the path with the minimal a star score
        try:
            minimal_path = list(all_a_star_score_dic.keys())[
                list(all_a_star_score_dic.values()).index(all_a_star_score.get(0))]

        except:
            print("no path found")
            return []
            break

        del all_a_star_score_dic[minimal_path]

        # start from the last node in the minimal a star score path
        last_minimal_path = minimal_path[-1]
        # print(last_minimal_path,"path")
        all_cells[last_minimal_path] = RED
        all_cells[start] = ORANGE
        draw()


        if last_minimal_path[0] == goal[0] and last_minimal_path[1] == goal[1] or last_minimal_path[0] == goal[0]-1 and last_minimal_path[1] == goal[1] or last_minimal_path[0] == goal[0]+1 and last_minimal_path[1] == goal[1]or last_minimal_path[0] == goal[0] and last_minimal_path[1] == goal[1]-1or last_minimal_path[0] == goal[0] and last_minimal_path[1] == goal[1]+1:

            for item in minimal_path:
                all_cells[start] = ORANGE
                all_cells[item] = PURPLE
                draw()
            return tuple(minimal_path)

        # find all neighbors of d[minimal_path] e.g(1,1)
        for neighbors in grid[last_minimal_path]:

            temp = (neighbors[0], neighbors[1])

            if (temp == goal):

                one_more_node = ()

                for item in minimal_path:
                    one_more_node = one_more_node + (item,)

                one_more_node = one_more_node + ((tuple(neighbors)),)

                loop = False
                return one_more_node

            # calculate a star score of that neighbor (heuristics of itself + cost between different grid, assume all is 1)
            a_star_score = len(minimal_path) + heuristics[temp]

            # update a star score of that neighbor if the new a_star_score is smaller
            if a_star_score < a_star_score_list[temp]:

                a_star_score_list[temp] = a_star_score

                # put all a star score of different neighbours into the priority queue to find the minimal
                all_a_star_score.put(a_star_score)

                # compute a path that made up of different points with the calculated a star score

                one_more_node = ()

                for item in minimal_path:
                    one_more_node = one_more_node + (item,)

                one_more_node = one_more_node + ((tuple(neighbors)),)
                all_a_star_score_dic[(one_more_node)] = a_star_score


def main(win, width):
    ROWS = 50

    for i in range(50):
        for w in range(50):
            all_cells[(i,w)] =WHITE
    run = True

    start_flag = True
    start = None
    end_flag = True
    end = None

    while run:
        draw(win, all_cells, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]:  # LEFT
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                print(row,col)

                if start_flag:
                    all_cells[(row, col)] = ORANGE
                    start = (row,col)
                    start_flag = False

                elif end_flag:
                    all_cells[(row, col)] = BLUE
                    end = (row, col)
                    end_flag = False
                else:
                    all_cells[(row,col)] = BLACK



            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:

                    unoccupied_cells = []
                    for item in all_cells:

                        if all_cells[item] != BLACK:

                            unoccupied_cells.append(item)



                    for item in unoccupied_cells:

                        update_neighbors(item[0],item[1],all_cells)

                    p = a_star_algorithm(lambda: draw(win, all_cells, ROWS, width), neighbors, start, end,50,50)
                    print(p)



    pygame.quit()

In [26]:
def main(win, width):
    ROWS = 50

    for i in range(50):
        for w in range(50):
            all_cells[(i,w)] =WHITE
    run = True

    start_flag = True
    start = None
    end_flag = True
    end = None

    while run:
        draw(win, all_cells, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]:  # LEFT
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                #print(row,col)

                if start_flag:
                    all_cells[(row, col)] = ORANGE
                    start = (row,col)
                    start_flag = False

                elif end_flag:
                    all_cells[(row, col)] = BLUE
                    end = (row, col)
                    end_flag = False
                else:
                    all_cells[(row,col)] = BLACK



            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:

                    unoccupied_cells = []
                    for item in all_cells:

                        if all_cells[item] != BLACK:

                            unoccupied_cells.append(item)



                    for item in unoccupied_cells:

                        update_neighbors(item[0],item[1],all_cells)

                    p = a_star_algorithm(lambda: draw(win, all_cells, ROWS, width), neighbors, start, end,50,50)
                    print(p)



    pygame.quit()

In [27]:
main(WIN, WIDTH)

((9, 9), (9, 9), (10, 9), (11, 9), (12, 9), (12, 10), (12, 11), (12, 12), (12, 13), (12, 14), (12, 15), (12, 16), (12, 17), (12, 18), (13, 18), (14, 18), (15, 18), (16, 18), (17, 18), (18, 18), (19, 18), (20, 18), (21, 18), (21, 19), (21, 20), (21, 21))
